# FT用データ生成スクリプト

In [1]:
# テキスト生成AIに関係するパッケージインストール
# -U: 最新版のインストール
!pip install -U openai \
    langchain==0.3.12 \
    langchain-community \
    langchain-core \
    langchain-google-genai \
    langchain-openai \
    langgraph \
    python-dotenv

# rag用パッケージのインストール
!pip install -U chromadb \
    langchain-chroma \
    pypdf \
    pdfminer.six

# coquiインストール
!pip install TTS[ja]

# 文字起こし用パッケージインストール
!pip install git+https://github.com/openai/whisper.git

!pip install librosa

#mecabと日本語辞書をインストール
!pip install mecab-python3 \
 unidic-lite

  Using cached openai-2.3.0-py3-none-any.whl.metadata (29 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_google_genai-2.1.12-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_openai-0.3.35-py3-none-any.whl.metadata (2.4 kB)
  Using cached langgraph-0.6.10-py3-none-any.whl.metadata (6.8 kB)
  Using cached langsmith-0.2.11-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.78-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.77-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_core-0.3.76-py3-none-any.whl.metadata (3.7 kB)
INFO: pip is still looking at m

## テキスト対話データ生成

In [2]:
import os
from typing import Literal
import ast

from dotenv import load_dotenv
from langchain import hub
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader, PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv()

/home1/s1f102201582/anaconda3/envs/py10_jupy_only/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
#config
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 2.0
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

# 生成する音声のサンプリングレート
sr = 16000

#対話音声データの個数を指定
gen_dial_num = 1

# すでに作成した対話データを削除するかどうか
IS_REMOVE_EXIST_FILE = False

# ftに使うjsonとaudioの出力フォルダパス
home_dir = expanduser("~")
json_dir_path = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/mecab_validate/transcription")
audio_dir_path = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/mecab_validate/audio")

In [ ]:
exist_check_paths = [
    json_dir_path,
    audio_dir_path,
]

for p in exist_check_paths:
    if not os.path.isdir(p):
        os.makedirs(p)

In [4]:
# client作成
llm = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

In [5]:
loader = DirectoryLoader(
    "../../mental_docs/",
    glob="*.pdf",
    show_progress=True,
    loader_cls=PyPDFLoader,
    # loader_cls=PDFMinerLoader
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.26it/s]

Loaded 159 documents


In [6]:
# Debug
# for doc in docs:
#     print("-------------------------------------------------")
#     print(doc.metadata)
#     print(len(doc.page_content))
#     print(doc.page_content[:100])

In [7]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)

# 埋め込み
embedding = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-small"
)

#ベクトルデータベースのChromaDBaに保存
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding
)

In [8]:
# vectorstoreから必要な情報を読み出す
retriever = vectorstore.as_retriever()

# ユーザーが与えるプロンプトに加えて、
# rag_promptを追加してLLMに与えるように設定
rag_prompt = hub.pull("rlm/rag-prompt")

In [9]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [10]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [11]:
#promptを作成
prompt_txt = """臨床心理士が行うメンタルヘルスケアカウンセリングをシミュレーションし、その対話内容に相槌を含め、話し言葉のまま文字起こししてください。
会話は中途半端で終わらせず、きりが良い会話にしてください。
相槌は実際の対話を想定して細かく入れてください。
語感は固くならないようにしてください。

形式は以下のようにしてください。Aがカウンセラーで、Bがカウンセリングを受ける人です。
以下の例は３回しか言葉を交わしていませんが、500文字程度の会話になるようにしてください。
カウンセリングで話す悩みは仕事以外にも考えうる様々なテーマすべて取り扱ってください。
文字列内に「A: 」のような誰が話したのかを明記する必要はありません。

[
 "Aが話す言葉",
 "Bが話す言葉",
 "Aが話す言葉",
 ...
]
"""

In [12]:
# テキスト対話生成関数
def gen_txt_dialogue():
    return rag_chain.invoke(prompt_txt)

In [13]:
# テキストからリスト変換関数
def txt_to_lst(txt):
    return ast.literal_eval(txt)

In [14]:
# DEBUG
# txt_dialogue = gen_txt_dialogue()
# print(txt_dialogue)
# lst_dialogue = txt_to_lst(txt_dialogue)
# print(lst_dialogue)

## テキスト対話データを音声対話データに変換 

In [15]:
from collections import defaultdict

import torch
import librosa
import numpy as np
import soundfile as sf
from TTS.utils.radam import RAdam
from TTS.api import TTS

model_name = "tts_models/ja/kokoro/tacotron2-DDC"

# デバイス設定
device = "cuda" if torch.cuda.is_available() else "cpu"

# PyTorchのtorch.loadのweights_only引数の仕様変更によるエラー防止
# 許可リストに追加 (これをしないとエラーが出る)
torch.serialization.add_safe_globals([RAdam])
torch.serialization.add_safe_globals([defaultdict])
torch.serialization.add_safe_globals([dict])

# モデルをダウンロードしていない場合は例外が出る
while True:
    try:
        model_path = os.path.expanduser("~/.local/share/tts/tts_models--ja--kokoro--tacotron2-DDC/model_file.pth")

        # 安全にロード（weights_only=TrueにしてもOK）
        model1 = torch.load(model_path, weights_only=True)
        break
        
    except Exception:
        # モデルをダウンロード
        TTS(model_name=model_name)

# モデル名指定してTTSインスタンス作成
tts = TTS(model_name=model_name).to(device)

In [17]:
def tts_coqui(text: str):
    wav = tts.tts(text)
    return wav

In [18]:
def lst_to_audio_dialogue(lst_dialogue):
    # 音声ファイルを順番に生成（ファイルは不要なのでwave配列で持つ）
    wav_data = []
    for i, text in enumerate(lst_dialogue):
        speaker = "A" if i%2==0 else "B"
        wav = tts_coqui(text)
        
        # numpy配列でなければ変換（torch.Tensorやlistの場合にも対応）
        if not isinstance(wav, np.ndarray):
            wav = np.array(wav, dtype=np.float32)
        
        if sr != tts.synthesizer.output_sample_rate:
            wav = librosa.resample(wav, orig_sr=tts.synthesizer.output_sample_rate, target_sr=sr)
        wav_data.append(wav)
    
    # 最終的な音声長を決定
    max_len = sum([len(w) for w in wav_data])
    
    # ステレオ音声用（2チャンネル×最大長）の空配列をゼロ初期化で作成
    stereo = np.zeros((2, max_len), dtype=np.float32)
    
    pos = 0
    for i, wav in enumerate(wav_data):
        ch = i%2  # 0:左(A), 1:右(B)
        stereo[ch, pos:pos+len(wav)] += wav
        pos += len(wav)
    
    # 転置し(-1,2)する
    stereo = stereo.T
    return stereo

## 音声対話データを文字起こししたjsonファイルを生成

In [27]:
import whisper
import torch
import soundfile as sf
import numpy as np

device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')

# Whisperモデルをロード
model = whisper.load_model("large-v3-turbo", device=devide)

Number of GPUs available: 4


In [20]:
import MeCab

def tokenize_text(text):
    tokens = []
    try:
        # MeCabのタガーを初期化
        tagger = MeCab.Tagger()

        # MeCabは内部でShift-JISやEUC-JPを期待することがあるため、
        # UnicodeDecodeErrorを避けるために明示的にUTF-8でエンコード・デコードする
        # parseToNodeは、より詳細な情報をノードオブジェクトとして取得できるメソッド
        node = tagger.parseToNode(text)
        while node:
            if node.surface:
                tokens.append(node.surface)
            
            # 次のノードへ
            node = node.next

    except RuntimeError as e:
        print(f"MeCabの実行中にエラーが発生しました: {e}", file=sys.stderr)
        
    return tokens

In [21]:
def generate_prompt(correct_txt, check_txts):
    prompt = f"""テキストAを正解としてテキストAを区切ったリストBの間違い箇所を修正してください。
出力はリスト文字列のみにすること。
文字列の区切りやリストの個数は変えないこと。
例: 
テキストA: "最近暑いですね。何か飲み物を買っていきましょうか。"
リストB: ["最近", "熱いで", "ね。", "何か", "飲み", "を勝手", "いきま", "章か。"]
出力: ["最近", "熱いで", "すね。", "何か", "飲み物", "を買って", "いきま", "しょうか。"]

ターゲット:
テキストA: {correct_txt}
リストB: [{", ".join(check_txts)}]
"""
    return prompt

In [22]:
def validate_text(correct_txts, target_txts):
    checked_pos = 0
    correct_txt = "".join(correct_txts)
    prompt = generate_prompt(correct_txt, target_txts)
    result = llm.invoke(prompt)
    ai_modified_cnt = 0
    while ai_modified_cnt < 10:
        try:
            ai_modified_txts = txt_to_lst(result.content)
            # リストの個数が変わっていないかチェック
            if len(ai_modified_txts) != len(target_txts):
                raise ValueError("AIの修正が間違っています。")

            # 修正後のテキストが正しいかチェック
            checked_pos = 0
            for i, txt in enumerate(ai_modified_txts):
                txt_length = len(txt)
                correct_txt_fragment = correct_txt[checked_pos:checked_pos+txt_length]
                if correct_txt_fragment != txt:
                    ai_modified_cnt += 1
                    raise ValueError("AIの修正が間違っています。")
                checked_pos += txt_length
            break
        except:
            pass
            
    if ai_modified_cnt == 10:
        print("AIが10回連続で修正を間違えたため、強制終了します。")
            
    return ai_modified_txts

In [30]:
def transcribe_channel(channel, sr, speaker_label, correct_txts):
    result = model.transcribe(
        channel.astype(np.float32), 
        language='ja', 
        word_timestamps=True
    )

    whisper_words = []
    for segment in result["segments"]:
        for word in segment["words"]:
            whisper_words.append(word)

    # テキストをaiで修正
    target_words = [word["word"] for word in whisper_words]
    validated_words = validate_text(result["text"], target_words)
    for i in range(whisper_words):
        whisper_words[i]["word"] = validated_words[i]

    # Mecab
    mecab_words = tokenize_text(result["text"])
    whisper_pos = 0
    mecab_pos = 0
    whisper_skip = 0
    mecab_skip = 0
    words_json = []
    word_start = ""
    
    while whisper_pos <= len(whisper_words)-1:
        if whisper_skip == 0:
            whisper_text = whisper_words[whisper_pos]["word"]
            word_start = whisper_words[whisper_pos]["start"]
        else:
            whisper_text = "".join([
                whisper_word["word"] for whisper_word in whisper_words[whisper_pos:whisper_pos+whisper_skip+1]
            ])
        
        if mecab_skip == 0:
            mecab_text = mecab_words[mecab_pos]
        else:
            mecab_text = "".join(mecab_words[mecab_pos:mecab_pos+mecab_skip+1])
        whisper_text = whisper_text.strip()
        mecab_text = mecab_text.strip()
        
        if whisper_text == mecab_text:
            words_json.append({
                'speaker': speaker_label,
                'word': whisper_text,
                'start': word_start,
                'end': whisper_words[whisper_pos+whisper_skip]["end"]
            })

            whisper_pos = whisper_pos + whisper_skip + 1
            mecab_pos = mecab_pos + mecab_skip + 1
            whisper_skip = 0
            mecab_skip = 0
            # print(f"whisper_text: {whisper_text}")
            # print(f"mecab_text: {mecab_text}")
            # print("words_jsonに追加しました。")
            # print(f"start: {word_start}")
            # print("------------------------------------------------------")
    
        elif len(whisper_text) < len(mecab_text):
            whisper_skip += 1
            # print("whisper_skipに+1しました。")
            # print(f"whisper_text: {whisper_text}")
            # print(f"mecab_text: {mecab_text}")
            # print(f"start: {word_start}")
            # print("------------------------------------------------------")
        
        elif len(whisper_text) > len(mecab_text):
            mecab_skip += 1
            # print("mecab_skipに+1しました。")
            # print(f"whisper_text: {whisper_text}")
            # print(f"mecab_text: {mecab_text}")
            # print(f"start: {word_start}")
            # print("------------------------------------------------------")
            
    return words_json

def transcribe_audio_dialogue(audio_path, correct_txts):
    # ステレオ分離: speaker A=左(0), B=右(1)と仮定
    audio, sr = sf.read(audio_path)    # (samples, channels)
    channel_A = audio[:,0]
    channel_B = audio[:,1]
    correct_txts_A = correct_txts[0::2]
    correct_txts_B = correct_txts[1::2]
    
    # 両チャンネルを transcribe
    json_A = transcribe_channel(channel_A, sr, "A", correct_txts_A)
    json_B = transcribe_channel(channel_B, sr, "B", correct_txts_B)
    
    # 発話時間でソート（複数話者の時系列並び用）
    full_json = json_A + json_B
    full_json_sorted = sorted(full_json, key=lambda x: x['start'])
    return full_json_sorted

In [24]:
import re

def get_continuous_num_on_wav_file():
    wav_file_pattern = r"^(\d+)\.wav$"
    max_num = -1
    for file in os.listdir(audio_dir_path):
        if not os.path.exists(os.path.join(audio_dir_path, file)):
            continue
        if not re.match(wav_file_pattern, file):
            continue
    
        match_obj = re.match(wav_file_pattern, file)
        get_number = int(match_obj.groups()[0])
    
        if max_num < get_number:
            max_num = get_number
    return max_num

In [25]:
from glob import glob

if IS_REMOVE_EXIST_FILE:
    max_num = -1

    # jsonファイルがある場合、削除
    json_file_paths = glob(os.path.join(json_dir_path, "*"))
    for file_path in json_file_paths:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_path} を削除しました。")

    # 音声ファイルがある場合、削除
    audio_file_paths = glob(os.path.join(audio_dir_path, "*"))
    for file_path in audio_file_paths:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_path} を削除しました。")
    
else:
    max_num = get_continuous_num_on_wav_file()

In [ ]:
for i in range(max_num+1, gen_dial_num+max_num+1):

    # 生成AIがリストのフォーマットでテキストを出力できない場合もあるので例外処理
    while True:
        try:
            txt_dialogue = gen_txt_dialogue()
            lst_dialogue = txt_to_lst(txt_dialogue)
            break
        except SyntaxError:
            pass
        
    stereo = lst_to_audio_dialogue(lst_dialogue)

    wav_name = f"{i}.wav"
    audio_file_path = os.path.join(audio_dir_path, wav_name)
    sf.write(audio_file_path, stereo, sr)

    json_data = transcribe_audio_dialogue(audio_file_path)

    json_name = f"{i}.json"
    json_file_path = os.path.join(json_dir_path, json_name)
    
    # JSON出力
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

 > Text splitted to sentences.
['こんにちは。', '今日は、どうされましたか？', '何か、お話ししたいこと、ありますか？']
 > Processing time: 0.2989695072174072
 > Real-time factor: 0.04446070488119017
 > Text splitted to sentences.
['ええと…何から話したらいいか、ちょっと、まだまとまらなくて。', 'でも、最近、すごく疲れてるなって感じてて。']
 > Processing time: 0.28389883041381836
 > Real-time factor: 0.028934186929747332
 > Text splitted to sentences.
['なるほど、すごくお疲れを感じているんですね。', 'うんうん。']
 > Processing time: 0.1669466495513916
 > Real-time factor: 0.03312314301943731
 > Text splitted to sentences.
['なんか、漠然とした不安っていうか…これからどうなるんだろうとか、周りの友達はみんな頑張ってるのに、私だけ立ち止まってるみたいで。']
 > Processing time: 0.3030862808227539
 > Real-time factor: 0.028875961338324072
 > Text splitted to sentences.
['周りの方が頑張っている中で、ご自身が立ち止まっているように感じる、漠然とした不安がある、ということなんですね。', 'ええ。']
 > Processing time: 0.28910326957702637
 > Real-time factor: 0.028650973923906187
 > Text splitted to sentences.
['そうなんです。', '具体的に何かあったわけじゃないんですけど、朝起きるのが億劫だったり、前は楽しかったことも、いまはあんまり気持ちが乗らなくて。']
 > Processing time: 0.3667008876800537
 > Real-ti